In [1]:
%matplotlib inline
from scipy import *
from numpy import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from ipywidgets import interactive_output, FloatSlider, HBox, VBox, Layout
from IPython.display import display

In [2]:
def interact_ellipse_compare():
    
    ## Set up the figure and the comparison plot of ellipses.
    fig = plt.figure(figsize=(8,8))
    
    real_ellipse, = plt.plot([], [], color='k', linewidth=2, label=r"Equation 4")
    comp_ellipse, = plt.plot([], [], color='r', linestyle='dashed', linewidth=2, label=r"Equation 20")

    hw=0.06
    hl=0.10
    real_arrow = plt.arrow([],[],[],[],
                  head_width = hw, head_length = hl, fc = 'black', ec = 'black')
    comp_arrow = plt.arrow([],[],[],[],
                  head_width = hw, head_length = hl, fc = 'red', ec = 'red', linestyle='dashed')

    ## Set up the figure axes, etc.
    plt.xlim(-1.0, 1.0)
    plt.ylim(-1.0, 1.0)
    plt.legend(loc="upper left")
    
    ## Place the figure spines at the origin
    ax = plt.gca()
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.spines['bottom'].set_position(('data',0))
    ax.yaxis.set_ticks_position('left')
    ax.spines['left'].set_position(('data',0))

    # Turn off tick labels
    ax.set_yticklabels([])
    ax.set_xticklabels([])

    x_inches = 8
    y_inches = 10
    plt.rcParams['figure.figsize'] = [x_inches,y_inches]
    plt.close()

    # initialise some arrays used by plot_ellipse_compare
    N = 60
    phi = linspace(0, 2*math.pi, N)
    cosph = cos(phi)
    sinph = sin(phi)
    j = complex(0,1)
    expph = exp(j*phi)
    
    # r, chi, and psi shared by plot_ellipse_compare and 
    r = 1.0
    chi = 25.0 * math.pi/180.0
    psi = 45.0 * math.pi/180.0
    
    ## Callback function: plot Equations 4 and 20
    def plot_ellipse_compare(r_val,chi_deg,psi_deg):
        
        r = r_val
        chi = chi_deg*math.pi/180.0
        psi = psi_deg*math.pi/180.0
        
        ## Equation 4
        xprime = r*cos(chi)*sinph
        yprime = r*sin(chi)*cosph
        
        ## Inverse of Equations 2 and 3
        x = xprime*cos(psi) - yprime*sin(psi)
        y = xprime*sin(psi) + yprime*cos(psi)
        
        real_ellipse.set_data(x, y)

        # draw real arrow at zero phase
        q=0
        real_arrow.set_data(x=x[q], y=y[q], 
                            dx=x[q+1]-x[q], dy=y[q+1]-y[q])

        ## Equation 20
        
        e_x = -j * r * complex( cos(psi)*cos(chi), -sin(psi)*sin(chi) ) * expph
        e_y = -j * r * complex( sin(psi)*cos(chi),  cos(psi)*sin(chi) ) * expph

        x = real(e_x)
        y = real(e_y)
        
        comp_ellipse.set_data(x, y)
        # draw complex arrow at 180 deg phase
        q=N//2
        comp_arrow.set_data(x=x[q], y=y[q], 
                            dx=x[q+1]-x[q], dy=y[q+1]-y[q])

        display(fig)

    ## Create slider bars
    r_val   = FloatSlider(min=0, max=1.0, step=0.1, value=1.0, description="$r$")
    chi_deg = FloatSlider(min=-45, max=45, step=9, value=25.0, description="$\chi$ (deg)")
    psi_deg = FloatSlider(min=-90, max=90, step=9, value=45.0, description="$\psi$ (deg)")

    ## Attach slider bars to plot_ellipse_compare
    w = interactive_output(plot_ellipse_compare,{'r_val':r_val, 'chi_deg':chi_deg, 'psi_deg':psi_deg})

    # Vertical layout of sliders
    box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    justify_content='center',
                    border='none',
                    width='50%')
    
    ui = VBox([r_val, chi_deg, psi_deg], layout=box_layout)
    
    # Horizontal Layout puts sliders beside plot
    display(HBox([w,ui]))


In [3]:
interact_ellipse_compare()